In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import pathlib
from pathlib import Path
from PIL import Image

In [11]:
## Funções para aumento de dados:
def resize(image, mask, height, width):
  nimage = tf.image.resize(image, [height, width])
  nmask = tf.image.resize(mask, [height, width])
  return nimage, nmask

def random_crop(image, mask, height, width):
  stacked_image = tf.stack([image, mask], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, height, width, 1])
  return cropped_image[0], cropped_image[1]

def random_rot(image, mask):     # Angulo [90, 180, 270]
    kAngles = np.random.randint(1, high=4, dtype=int)
    nimage = tf.image.rot90(image, k =kAngles)
    nmask =  tf.image.rot90(mask, k= kAngles)
    return nimage, nmask

def random_jitter(image, mask):
  #print('Jitter..')
  if tf.random.uniform(()) > 0.5:  
     # Random cropping  128x128
     hasMask = 0
     while(hasMask <= 20):
         nimage, nmask = random_crop(image, mask, 128, 128)
         hasMask = tf.math.reduce_sum(nmask)
  else:
     nimage, nmask = resize(image, mask, 128, 128)    
 
  if tf.random.uniform(()) > 0.5:  
     # Random mirroring rotate angle [90, 180, 270]
     nimage, nmask = random_rot(nimage, nmask)

  if tf.random.uniform(()) > 0.5:  
    # Random mirroring left_rigth
    nimage = tf.image.flip_left_right(nimage)
    nmask = tf.image.flip_left_right(nmask)
  else:
     # Random mirroring up_down
     nimage = tf.image.flip_up_down(nimage)
     nmask = tf.image.flip_up_down(nmask) 
      
  return nimage, nmask

In [5]:
# Função para gerar um label em 2 canais(background e foreground) a partir de uma máscara
def geraLabel(mask):
    label = np.zeros((mask.shape[0], mask.shape[1], 2), dtype=np.uint8)
    back = np.logical_not(mask)
    fore = mask
    label[:,:,0] = back
    label[:,:,1] = fore
 
    return label

In [6]:
# Testa GeraLabel
x = tf.constant([[0,1], [0,1]])
label = geraLabel(x)
x, label, label.shape,   label[:,:,0], label[:,:,1]

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[0, 1],
        [0, 1]])>,
 array([[[1, 0],
         [0, 1]],
 
        [[1, 0],
         [0, 1]]], dtype=uint8),
 (2, 2, 2),
 array([[1, 0],
        [1, 0]], dtype=uint8),
 array([[0, 1],
        [0, 1]], dtype=uint8))

In [15]:
# Script para aplicar o aumento de dados
 
PATH_aumento = "D:/baseISBI/treinoOriginais";
PATH_dest_aumento = "D:/baseISBI/"
num = 0     
paths = Path(PATH_aumento).glob('*[0-9].*')
for path in paths:
    #print(path, path.stem)
    with Image.open(path) as im:
     with Image.open("D:/baseISBI/treinoOriginais/"+path.stem+"_GT.png") as ma:        
        image = tf.convert_to_tensor(np.array(im), dtype=tf.float32)
        mask = tf.convert_to_tensor(np.array(ma), dtype=tf.float32)
        
        ims, mas = resize(image[..., tf.newaxis], mask[..., tf.newaxis], 128, 128)
        
        filenameImg = os.path.join(PATH_dest_aumento,"trainX", path.stem+".png")
        filenameMask = os.path.join(PATH_dest_aumento,"masks", path.stem+"_GT.png")
        filenameLabel = os.path.join(PATH_dest_aumento,"trainY", path.stem+"_GT.png")
                
        ims = tf.cast(tf.squeeze(ims), dtype=tf.uint8)
        mas = tf.cast(tf.squeeze(mas), dtype=tf.uint8)  # tipo logical do Matlab  
        lab = geraLabel(mas.numpy())
                 
        print(lab.shape, lab.dtype, tf.math.reduce_max(mas), tf.math.reduce_max(lab))
        ims = Image.fromarray(ims.numpy())
        mas = Image.fromarray(mas.numpy()*255)
        lab = Image.fromarray(lab*255)
         
        ims.save(filenameImg)
        mas.save(filenameMask)
        lab.save(filenameLabel)
        num = num + 1
        #Para cada imagem original, gera 20 aumentos
        for i in range(20):
                fileImg = path.stem + "_"+str(i+1) + ".png"
                fileMask = path.stem + "_"+str(i+1) + "_GT.png"
                fileLabel = path.stem + "_"+str(i+1) + "_GT.png"
                
                filenameImg = os.path.join(PATH_dest_aumento, 'trainX', fileImg)
                filenameMask = os.path.join(PATH_dest_aumento, 'masks', fileMask)
                filenameLabel = os.path.join(PATH_dest_aumento,"trainY", fileLabel)
                
                nimage, nmask = random_jitter(tf.expand_dims(image, axis=2), tf.expand_dims(mask, axis=2))
                 
                nimage = tf.cast(tf.squeeze(nimage), dtype=tf.uint8).numpy()
                nmask = tf.cast(tf.squeeze(nmask), dtype=tf.uint8).numpy() 
                label = geraLabel(nmask)              
                 
                nimage = Image.fromarray(nimage)
                nmask = Image.fromarray(nmask*255)
                label = Image.fromarray(label*255)
                
                nimage.save(filenameImg)
                nmask.save(filenameMask)     
                label.save(filenameLabel)
                num = num + 1    

(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(1, shape=(), dtype=uint8) tf.Tensor(1, shape=(), d

In [16]:
num

945

In [19]:
## Testa labels gerados
#paths = Path("D:/baseISBI/treinoOriginais").glob('*_GT.png')    
flag = 0
paths = Path("D:/baseISBI/trainY/").glob('*_GT.png')    
for path in paths:
   with Image.open(path) as f:
         img = np.array(f)
         image = tf.convert_to_tensor(img, dtype=tf.uint8)
         image = tf.cast(image, dtype=tf.uint8)   
         if (int(tf.math.reduce_max(image)) != 255):
             print("Erro! Sim", path, int(tf.math.reduce_max(image)))
             flag = 1
             break

print(flag)

0


In [21]:
## Compara mask e label
with Image.open("D:/baseISBI/masks/SYN44_GT.png") as f:
         f.show()
         print(f, np.array(f).shape)
         mask = np.array(f)
         m = Image.fromarray(mask*255)
         m.show()
         if (int(tf.math.reduce_max(mask)) > 0):
             print("Sim", int(tf.math.reduce_max(mask)))
         else:
             print("Não")
                
                
with Image.open("D:/baseISBI/trainY/SYN44_GT.png") as f:
         f.show()
         print(f, np.array(f).shape)
         label = np.array(f)
         if (int(tf.math.reduce_max(label)) > 0):
             print("Sim", int(tf.math.reduce_max(label)))
         else:
             print("Não")
x = 60
y = 60
mask.shape, label.shape, mask[x:x+30,y:y+20], label[x:x+30,y:y+20,1], np.sum(mask)/255, np.sum(label[:,:,1])/255
 

<PIL.PngImagePlugin.PngImageFile image mode=L size=128x128 at 0x20373DB9D30> (128, 128)
Sim 255
<PIL.PngImagePlugin.PngImageFile image mode=LA size=128x128 at 0x20373DB9A90> (128, 128, 2)
Sim 255


((128, 128),
 (128, 128, 2),
 array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0],
        [  0, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0],
        [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0],
        [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0],
        [255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0],
        [  0, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 

In [23]:
### Gera csv para arquivos de treinamento no formato (nomeImagem, nomeLabelDaImagem):
import csv

header = ['features', 'label']
file = os.path.join(PATH_dest_aumento, 'trainISBI2014.csv')

with open(file, 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    paths = Path(PATH_dest_aumento,"trainX").glob('*.png')    
    for path in paths:
       data = [path.stem+".png", path.stem+"_GT.png"]
       writer.writerow(data)
  

In [32]:
# Rezise no conjunto labels para conjunto validação (sem aumento de dados)
PATH_de = "D:/baseISBI/valMasks"
PATH_para ="D:/baseISBI/valY"
num = 0     
flag = 0
paths = Path(PATH_de).glob('*_GT.png')
for path in paths:
    #print(path, path.stem)
    with Image.open(path) as mask:
        filename = path.stem + ".png"
        mask = tf.convert_to_tensor(np.array(mask), dtype=tf.uint8)
        filenameLabel = os.path.join(PATH_para,filename)
        label = geraLabel(mask.numpy())                 
        print(label.shape, label.dtype, tf.math.reduce_max(label), tf.math.reduce_max(mask))
        if (int(tf.math.reduce_max(label)) != 255):
             print("Erro! Sim", path, int(tf.math.reduce_max(label)))
             flag = 1
             break
        label = Image.fromarray(label)         
        label.save(filenameLabel)
        num+=1

(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape=(), dtype=uint8) tf.Tensor(255, shape=(), dtype=uint8)
(128, 128, 2) uint8 tf.Tensor(255, shape

In [33]:
num

90

In [35]:
### Gera csv para arquivos de validação no formato (nomeImagem, nomeLabelDaImagem):
import csv
PATH_val = "D:/baseISBI/valX"
header = ['features', 'mask']
file = os.path.join(PATH_dest_aumento, 'valISBI2014.csv')

with open(file, 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    paths = Path(PATH_val).glob('*.png')    
    for path in paths:
       data = [path.stem+".png", path.stem+"_GT.png"]
       writer.writerow(data)